In [1]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from lohrasb.best_estimator import BaseModel
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score, accuracy_score, make_scorer
from lightgbm import *


2023-07-03 17:26:37,093 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-07-03 17:26:37,104 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-07-03 17:26:37,108 :: matplotlib :: interactive is False
2023-07-03 17:26:37,110 :: matplotlib :: platform is darwin
2023-07-03 17:26:37,142 :: graphviz._tools :: deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-03 17:26:37,145 :: graphviz._tools :: deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
2023-07-03 17:26:37,148 :: graphviz._tools :: deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
2023-07-03 17:26:37,151 :: graphviz._tools :: deprecate positional args: graphviz.backend.viewing.view(['quiet'])
2023-07-03 17:26:37,158 :: graphviz._tools :: deprecate positio

# Example: Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

# Part 1: Use BestModel in sklearn pipeline


In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#### Define labels


In [3]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

#### Train test split

In [4]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]


X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)


#### Find feature types for later use

In [5]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


#### Define estimator and set its arguments 


In [6]:
estimator = LGBMClassifier()
estimator_params = {
        "max_depth": [6,30],
        "learning_rate": [0.001,0.01, 0.1],
        "n_estimators": [100,200,1000],
        "boosting_type": ["gbdt"],

    }
    

In [7]:

kwargs = {  # params for fit method or fit_params 
            'fit_random_kwargs' :{
            'sample_weight':None,
            },
            # params for RandomSearchCV 
            'random_search_kwargs' : {
            'estimator':estimator,
            'param_distributions':estimator_params,
            'scoring' :'f1',
            'verbose':3,
            'n_jobs':-1,
            'cv':KFold(2),
            'n_iter':10,
            'refit':True,
            }
            }



In [8]:

obj = BaseModel().optimize_by_randomsearchcv(
    kwargs=kwargs
 )

#### Build sklearn pipeline

In [9]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj)

 ])


#### Run Pipeline

In [10]:
pipeline.fit(X_train,y_train.values.ravel())
y_pred = pipeline.predict(X_test)


Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV 1/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=6, n_estimators=100;, score=0.604 total time=   0.6s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=6, n_estimators=100;, score=0.542 total time=   0.6s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=6, n_estimators=100;, score=0.000 total time=   0.6s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=6, n_estimators=100;, score=0.000 total time=   0.6s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=30, n_estimators=200;, score=0.000 total time=   1.3s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=30, n_estimators=200;, score=0.000 total time=   1.3s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=30, n_estimators=100;, score=0.608 total time=   0.7s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.01, max_depth=30, n_estimators=100;, score=0.564 total time=   0.7s
[CV

#### Check performance of the pipeline

In [11]:
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))


F1 score : 
0.7156348373557189
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8158
           1       0.78      0.66      0.72      2588

    accuracy                           0.87     10746
   macro avg       0.84      0.80      0.82     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7686  472]
 [ 883 1705]]


# Part 2:  Use BestModel as a standalone estimator 

In [12]:
X_train, X_test, y_train, y_test =train_test_split(X, y, \
     test_size=0.33, stratify=y['label'], random_state=42)

#### Transform features to make them ready for model input

In [13]:
transform_pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model

 ])

#### Transform X_train and X_test

In [14]:
X_train=transform_pipeline.fit_transform(X_train,y_train)
X_test=transform_pipeline.transform(X_test)


#### Train model and predict

In [15]:
obj.fit(X_train,y_train.values.ravel())
y_pred = obj.predict(X_test)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV 1/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=30, n_estimators=100;, score=0.000 total time=   0.2s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=30, n_estimators=100;, score=0.000 total time=   0.3s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=30, n_estimators=100;, score=0.696 total time=   0.5s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=30, n_estimators=100;, score=0.705 total time=   0.5s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=30, n_estimators=200;, score=0.695 total time=   0.8s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.1, max_depth=30, n_estimators=200;, score=0.703 total time=   0.8s
[CV 1/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=6, n_estimators=200;, score=0.000 total time=   0.5s
[CV 2/2] END boosting_type=gbdt, learning_rate=0.001, max_depth=6, n_estimators=200;, score=0.000 total time=   0.5s
[CV 1

#### Check performance of the model

In [16]:
print('accuracy score score : ')
print(accuracy_score(y_test,y_pred))
print('F1 score : ')
print(f1_score(y_test,y_pred))
print('Classification report : ')
print(classification_report(y_test,y_pred))
print('Confusion matrix : ')
print(confusion_matrix(y_test,y_pred))

accuracy score score : 
0.8739065698864694
F1 score : 
0.7156348373557189
Classification report : 
              precision    recall  f1-score   support

           0       0.90      0.94      0.92      8158
           1       0.78      0.66      0.72      2588

    accuracy                           0.87     10746
   macro avg       0.84      0.80      0.82     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7686  472]
 [ 883 1705]]


In [17]:
obj.get_best_estimator()

LGBMClassifier(learning_rate=0.01, max_depth=30, n_estimators=1000)

In [18]:
obj.best_estimator

LGBMClassifier(learning_rate=0.01, max_depth=30, n_estimators=1000)

#### Get fitted randomized search object and its attributes

In [19]:
RandomSearchObj = obj.get_optimized_object()
RandomSearchObj.cv_results_

{'mean_fit_time': array([0.2189939 , 0.32954359, 0.44673145, 0.2136296 , 0.20653701,
        0.35000908, 0.22288311, 1.41575897, 1.46181953, 1.69025004]),
 'std_fit_time': array([0.00129414, 0.00983262, 0.0057677 , 0.00788939, 0.0002439 ,
        0.01158392, 0.00394905, 0.01689899, 0.01040566, 0.00490701]),
 'mean_score_time': array([0.1099602 , 0.21403801, 0.21604562, 0.11076188, 0.15031898,
        0.27176976, 0.11915851, 2.8319726 , 2.33446801, 3.25412738]),
 'std_score_time': array([0.00129187, 0.00154221, 0.00342155, 0.00320101, 0.00200307,
        0.00603008, 0.00146461, 0.07922554, 0.21713483, 0.02303445]),
 'param_n_estimators': masked_array(data=[100, 200, 200, 100, 100, 200, 100, 1000, 1000, 1000],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[30, 6, 30, 6, 30, 30, 30, 6, 30, 30],
              mask=[False, False, False, F